In [1]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import keras

Using plaidml.keras.backend backend.


In [2]:
train_data_directory = 'D:/data science/cats and dogs/train'

In [3]:
# image data preprocessing
image_size = (112,112)

batch_size = 64


class_mode = 'binary' # for multi-class classification problem, use: class_mode = 'category' 

from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode="nearest",
    validation_split=0.2 # set validation split
    ) 

train_generator = train_datagen.flow_from_directory(
    train_data_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode=class_mode, # for multi-class classification problem, use 'category'
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_directory, # same directory as training data
    target_size=image_size,
    batch_size=batch_size,
    class_mode=class_mode, # for multi-class classification problem, use 'category'
    subset='validation') # set as validation data

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [4]:
# build a model
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout

VGG_16 = Sequential()
VGG_16.add(Conv2D(input_shape=(112,112,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
VGG_16.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
VGG_16.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

VGG_16.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
VGG_16.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
VGG_16.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

VGG_16.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
VGG_16.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
VGG_16.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
VGG_16.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

VGG_16.add(Flatten())

VGG_16.add(Dense(1024,activation="relu"))
Dropout(0.5)
VGG_16.add(Dense(1024,activation="relu"))
Dropout(0.5)
VGG_16.add(Dense(1, activation="sigmoid"))

VGG_16.summary()

INFO:plaidml:Opening device "opencl_nvidia_geforce_rtx_3080.0"


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 112, 112, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 56, 56, 128)       73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 56, 56, 128)       147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 256)       295168    
__________

In [5]:
# compile model
from keras import optimizers

VGG_16.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [6]:
# Callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
checkpoint = ModelCheckpoint("vgg16_dog_cat.h5", 
                             monitor='val_acc', 
                             verbose=1, 
                             save_best_only=True, 
                             save_weights_only=False, 
                             mode='auto', 
                             period=1)
early = EarlyStopping(monitor='val_acc', 
                      min_delta=0, 
                      patience=10, 
                      verbose=1, 
                      mode='auto')
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=2,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.0000001)

In [ ]:
# fit/train model

nb_epochs = 1000

history = VGG_16.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs,
    callbacks=[checkpoint, early, learning_rate_reduction])

Epoch 1/1000
312/312 [==============================] - 139s 445ms/step - loss: 0.6504 - acc: 0.6147 - val_loss: 0.5884 - val_acc: 0.6821

Epoch 00001: val_acc improved from -inf to 0.68209, saving model to vgg16_dog_cat.h5
Epoch 2/1000
312/312 [==============================] - 118s 379ms/step - loss: 0.5783 - acc: 0.6957 - val_loss: 0.5434 - val_acc: 0.7184

Epoch 00002: val_acc improved from 0.68209 to 0.71840, saving model to vgg16_dog_cat.h5
Epoch 3/1000
312/312 [==============================] - 118s 379ms/step - loss: 0.5356 - acc: 0.7287 - val_loss: 0.5243 - val_acc: 0.7308

Epoch 00003: val_acc improved from 0.71840 to 0.73075, saving model to vgg16_dog_cat.h5
Epoch 4/1000
312/312 [==============================] - 119s 380ms/step - loss: 0.4889 - acc: 0.7618 - val_loss: 0.4725 - val_acc: 0.7753

Epoch 00004: val_acc improved from 0.73075 to 0.77532, saving model to vgg16_dog_cat.h5
Epoch 5/1000
312/312 [==============================] - 118s 378ms/step - loss: 0.4522 - acc: 0

In [ ]:
keras.__version__